***
#### Just Run to set up libraries and verify Stan exists

In [ ]:
#install.packages("cmdstanr", repos = c("https://mc-stan.org/r-packages/", getOption("repos")))
library(cmdstanr)
library(posterior)
set_cmdstan_path('/opt/conda/bin/cmdstan')
cmdstan_version()
source("/home/sagemaker-user/SharedTemplates/RCode/EstimationEcosystems.R") # Source helper functions

***
####  1) Create subfolder in Projects directory and import rds data file there 
Then specify in first three lines:  
><code style="color:yellow">my_folder</code> is name of folder you created under Projects   
><code style="color:yellow">my_csv_file</code> has name of csv file (in my_folder)   
><code style="color:yellow">out_prefix</code> alphanumeric descriptive prefix for all output files (e.g. a project code).  No "/", special symbols   
Advanced users can igore my_folder and set directories manually

In [ ]:
################    REQUIRED ENTRIES       #####################
my_folder <- "NewProject" # NAME OF your folder in Projects
my_csv_file <- "Data_Train.csv" 
out_prefix <- "my_tag" # Prefix for output files (descriptive string no /)

################    OPTIONAL       #####################
dir_work <- file.path("/home/sagemaker-user/Projects", my_folder) # path for output files
dir_data <- file.path("/home/sagemaker-user/Projects", my_folder) # path for data
dir_stan <- "/home/sagemaker-user/SharedTemplates/StanCode" # path for stan code
my_stan_code <- "BaseHB_wMNL.stan" # Optional: Specify other stan model here in subfolder StanCode

################ LEAVE BELOW ALONE #####################
message("Working directory for output set to: ", dir_work)
if (!file.exists(file.path(dir_data, my_csv_file))){
  message(paste0("Cannot find your data file: ", my_csv_file))    
} else {
  data_in <- read.csv(file.path(dir_data, my_csv_file), as.is=TRUE)
  message("Read data into R file data_in")
  str(data_in)
}
if (!file.exists(file.path(dir_stan, my_stan_code))){
  message(paste0("Cannot find Stan Code in: ", dir_stan))    
} else {
  cat("Compiling Stan code...")
  HB_model <- cmdstan_model(file.path(dir_stan,my_stan_code), quiet = TRUE, cpp_options = list(stan_threads = TRUE))
  cat("DONE")
}

#### 2) Specify Coding 

In [ ]:
idtaskdep <- data_in[,c(1,2,10)] # column numbers of id, task, dep

### SPECIFY CODING BELOW FOR EACH ATTRIBUTE ####
indcode_spec <- list()
indcode_spec[[1]] <- catcode(data_in, 4) 
indcode_spec[[2]] <- catcode(data_in, 5) 
indcode_spec[[3]] <- catcode(data_in, 6) 
indcode_spec[[4]] <- catcode(data_in, 7) 
indcode_spec[[5]] <- catcode(data_in, 8) 
indcode_spec[[6]] <- ordcode(data_in, 9, cut_pts = c(1,2,3,4,5), varout = "ppd") # Ordinal Code
#################################################

### Put coding together ###
message("Chosen id, task, dep:")
indcode_list <- make_codefiles(indcode_spec)
write.table(cbind(rownames(indcode_list$code_master), indcode_list$code_master), file = file.path(dir_work, paste0(out_prefix,"_code_master.csv")), sep = ",", na = ".", row.names = FALSE)
str(idtaskdep)
message("Code master file has the following coded parameters:")
print(colnames(indcode_list$code_master))

***
#### 3) Specify constraints

In [ ]:
### Specify constraints (sign only) ###
con_sign <- rep(0,ncol(indcode_list$code_master))
con_sign[18:21] <- -1 # Negative utilities for price slopes

In [ ]:
### Prepare data for Stan (Optional changes) ####
data_stan <- prep_file(idtaskdep, indcode_list)
data_stan$tag <- out_prefix 
data_stan$con_sign <- con_sign
data_stan$prior_cov <- data_stan$prior_cov * 1 # Prior covariance change
data_stan$df <- 2 # Degrees of freedom
data_stan$prior_alpha = rep(0, data_stan$P)
data_stan$a_sig = 10
data_stan$cov_block = matrix(1, data_stan$P, data_stan$P)
message("Stan file prepared")
str(data_stan)

***  
#### 4) Specify threads and modeling specs

In [ ]:
# Specify threads
threads <- list(tot_chains = 2, # Typically 2
                parallel_chains = 2, # Typically 2
                threads_per_chain = 2) # Depends on Cores, 8 to 16 is desirable

# Modeling parameters. Defaults are usually fine
data_model <- list(
  iter_warmup = 300, # warmup of 400 is plenty
  iter_sampling = 300, # sampling of 400 is plenty 
  splitsize = round(.5 + data_stan$T/(4 * threads[[3]])), # For multi-treading 1 is Stan automatic
  agg_model = NULL,
  tag = NULL
)
message("All Prep is Done")

***
#### Last cell below can just be run  
**-- Estimate Model, Save Results and Evaluate Convergence --**


In [ ]:
outname <- paste0(out_prefix, "_StanOut_", 
                  format(Sys.time(), '%Y%m%d-%H%M%S')) # edit as desired
message("ESTIMATING...")
cat("Optional lines to run in terminal to check progress:\n")
cat(paste0(
    "cd ", dir_work, "   # Change to your working directory and then:\n",
    "  awk 'END { print NR - 45 } ' '",outname,"-1.csv'", "                # Count lines in output\n",
    "  tail -n +45 '",outname,"-1.csv'  | cut -d, -f 1-300 > temp.csv", "  # Create temp.csv with first 300 columns\n"))

#####  Run Stan Model ###############
HB_model$sample(modifyList(data_stan, data_model),
                iter_warmup = data_model$iter_warmup,
                iter_sampling = data_model$iter_sampling,
                output_dir = dir_work,
                output_basename = outname,
                chains = threads[[1]],
                parallel_chains = threads[[2]],
                threads_per_chain = threads[[3]],
                save_warmup = TRUE,
                refresh = 50,
                seed = 271,
                init = .1,
                adapt_delta = .8,
                show_messages = FALSE,
                validate_csv = FALSE
)
cat("### ESTIMATION DONE ###\n")

### Read results from csv into R ###
cat("Reading draws from Stan csv output into R (large files take time)...")
nchains <- threads[[1]]
csv_name <- do.call(c, lapply(1:nchains, function(i) paste0(outname,"-",i,".csv")))
draws_beta <- read_cmdstan_csv(file.path(dir_work, csv_name), variables = "beta_ind", format = "draws_list", sampler_diagnostics = "accept_stat__")
cat("DONE")
                              
### Save output files and check convergence ###
draws_name <- paste0(out_prefix,"_draws_beta.rds")
util_name <- paste0(out_prefix,"_utilities_r.csv")
pdf_name <- paste0(out_prefix,"_trace_plots.pdf")
fit_name <-  paste0(out_prefix,"_fit_stats.csv")
message(paste0(
  "Saving post warm-up files for:\n",
  " respondent point estimates:    ", util_name,"\n",  
  " draws of utilities as R list:  ", draws_name,"\n",
  " convergence stats of mean:     ", fit_name, "\n",
  " PDF of detailed traceplots:    ", pdf_name,"\n",
  "\nShowing post warm-up:\n",
    " Acceptance rate across iterations (histogram)\n",
    " Traceplots of all mean utilities together (Sawtooth chart)"
))

hist(do.call(rbind,draws_beta$post_warmup_sampler_diagnostics)$accept_stat__, breaks = 30, main = "Acceptance Rate - Sampling", xlab = "", xlim = c(0,1))
saveRDS(modifyList(draws_beta,list(warmup_draws = NULL)), file.path(dir_work, draws_name)) # drop warmup
utilities <- matrix(
            Reduce("+",lapply(draws_beta$post_warmup_draws, colMeans))/nchains,
            data_stan$I, data_stan$P,
            byrow = TRUE) # First P entries are respondent 1, next P are resp 2
utilities_r <- utilities %*% t(data_stan$code_master)
write.table(cbind(id = data_stan$resp_id, utilities_r), file = file.path(dir_work, util_name), sep = ",", na = ".", row.names = FALSE)
                              
# Convergence charts saved as pdf and in fit_stats
fit_stats <- data.frame(
  variable = colnames(data_stan$ind),
  mean = NA,
  rhat = NA,
  ESS = NA
)
ndraws <- nrow(draws_beta$post_warmup_draws[[1]])
draws_beta_mu <- list() # Creates the mean of respondent utilities for each iteration, like alpha
for (chain_i in (1:nchains)){
  draws_beta_list <- as.matrix(draws_beta$post_warmup_draws[[chain_i]])
  draws_beta_mu[[chain_i]] <- t(sapply(1:ndraws, function(draw){
    beta_mu <- colMeans(matrix(draws_beta_list[draw,],
                               data_stan$I,data_stan$P, byrow = TRUE))
  }))
  matplot(1:nrow(draws_beta_mu[[chain_i]]), draws_beta_mu[[chain_i]],
          type = "l" , lty = 1, lwd = 1, main = paste0("Chain ", chain_i), xlab = "Iteration", ylab = "Mean Beta")   
} 
chain_cols <- c("red","blue","green","black")
pdf(file = file.path(dir_work, pdf_name),   # The directory you want to save the file in
    width = 7, # The width of the plot in inches
    height = 5) # The height of the plot in inches
for (i in 1:ncol(draws_beta_mu[[1]])){
  x <- sapply(1:length(draws_beta_mu), function(chain){
    draws_beta_mu[[chain]][,i]     
  }) # x is set of column i across draws_beta_mu
  fit_stats$mean[i] <- round(mean(x), 2)
  fit_stats$rhat[i] <- round(rhat(x),2)
  fit_stats$ESS[i] <- round(ess_basic(x),1)
  plot(x[,1], type = "l", col = chain_cols[1], ylim = c(min(x), max(x)),
                          xlab = "Sample Iteration", ylab = "Mean Beta",
                          main = paste(colnames(data_stan$ind)[i],
                                                    "| rhat = ", round(rhat(x),2),
                                                    "| ESS = ", round(ess_basic(x),1)
  ))
  for (chain in 2:nchains){
    lines(x[,2], type = "l", col = chain_cols[chain])
  }
}
dev.off()
write.table(fit_stats, file = file.path(dir_work, paste0(out_prefix,"_fit_stats.csv")), sep = ",", na = ".", row.names = FALSE)